In [1]:
%pip install -qr requirements.txt  # install


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import json
import webbrowser
import datetime


client_id = '5f6ad0b8fc02496783286ff99db144b8' #CHANGE FOR SPOTIFY DEVICE
client_secret = '13a94c2e1b594781ac7830bca5410b51' #CHANGE FOR SPOTIFY DEVICE

def add_to_queue(inp):

    scope = "user-modify-playback-state"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri='https://www.google.com/',
        scope=scope))
    print("Added to queue")
    
    search_str = inp
    artist_name=[]
    track_name = []
    track_id = []
    result = sp.search(search_str, type='track')
    for i, t in enumerate(result['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
    
    print(artist_name[0])
    print(track_name[0])


    #print(result)
    sp.add_to_queue(track_id[0])

def skip_song():

    scope = "user-modify-playback-state"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri='https://www.google.com/',
        scope=scope))
    print("skipping")

    # Skip to the next track
    sp.next_track()

#PARAMS TO CHANGE
accuracy_window = 60 #currently has to be smaller than 10 cuz thats the initial list size/2
decrease_to_skip = 0.6 #percentage decrease needed to skip song
seconds_to_skip = 30 #number of seconds after the last song was skipped so that the next song can skip

hand_values = [0 for i in range(120)] #fills the list with 0s
head_values = [0 for i in range(120)]

last_time_skipped = [datetime.datetime.now()]

def add_new_values(head=None, hand=None): #run every time a new head or hand count is found

    skip_bool = False
    #start_skip = False

    #SKIP FOR HANDS
    if hand is not None:
        hand_values.insert(0, hand)

        current_max = max(hand_values[0:accuracy_window])#sum(hand_values[0:accuracy_window])/accuracy_window
        previous_max = max(hand_values[accuracy_window:accuracy_window*2])#sum(hand_values[accuracy_window:accuracy_window*2])/accuracy_window
    
        if current_max < decrease_to_skip * previous_max:
            current_time = datetime.datetime.now()
            print("time passed:", (current_time - last_time_skipped[0]).total_seconds())
            if (current_time - last_time_skipped[0]).total_seconds() > seconds_to_skip:
                print("Skipping Song!")
                #skip_song()
                last_time_skipped[0] = datetime.datetime.now()
                skip_bool = True
                #start_skip = False
            else: 
                print("Not enough time to skip song!!")
                #start_skip = True

    if hand is not None: 
        hand_values.insert(0, hand)

    #SKIP FOR HEADS
    if head is not None:
        hand_values.insert(0, head)

        current_max = max(hand_values[0:accuracy_window])
        previous_max = max(hand_values[accuracy_window:accuracy_window*2])

        if current_max < decrease_to_skip * previous_max:
            current_time = datetime.datetime.now()
            print("time passed:", (current_time - last_time_skipped[0]).total_seconds())
            if (current_time - last_time_skipped[0]).total_seconds() > seconds_to_skip:
                print("Skipping Song!")
                #skip_song() MAKE SURE YOU UNCOMMENT TOMORROW
                last_time_skipped[0] = datetime.datetime.now()
                skip_bool = True
                #start_skip = False
            else: 
                print("Not enough time to skip song!!")
                #start_skip = True

    if head is not None: 
        head_values.insert(0, head)

    print("hand vals:", hand_values)
    print("head vals:", head_values)

    return skip_bool#, start_skip

In [1]:
# Webcam feed
  
from tkinter import *
import cv2
from PIL import Image, ImageTk
import time

import argparse
import os
import platform
import sys
from pathlib import Path

import torch

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode

# Directories
save_dir = increment_path(Path('runs/detect') / 'exp', exist_ok=False)  # increment run
(save_dir / 'labels' if False else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

#source = check_file("party_trimmed_recommendation.mp4")
model = DetectMultiBackend("partyai_model.pt")#, device='', dnn=False, data=None, fp16=False)
stride, names, pt = model.stride, model.names, model.pt
imgsz=(640, 640)
dataset = LoadStreams("0", img_size=imgsz, stride=stride, auto=pt, vid_stride=1) #load from webcam
bs = len(dataset)
dataset = iter(dataset) #[(path, im, im0s, vid_cap, s) for path, im, im0s, vid_cap, s in dataset]

vid_path, vid_writer = [None] * bs, [None] * bs

# Create a GUI app
app = Tk()
  
# Bind the app with Escape keyboard to
# quit app whenever pressed
app.bind('<Escape>', lambda e: app.quit())

# Label Creation
tracking_lbl = Label(app, text = "")
tracking_lbl.pack()

# Create a label and display it on app
label_widget = Label(app)
label_widget.pack()
label_widget.configure(height = 40, width=71)

# Run inference
model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))  # warmup

def inference(prev_headcount = 0, prev_handcount = 0):
    start_time = datetime.datetime.now()
    button1.pack_forget() #make button invisible
    try:
        path, im, im0s, vid_cap, s = next(dataset)  # get the next image from the iterator
    except StopIteration:
        return  # if there are no more images, do nothing
    
    windows, dt = [], (Profile(), Profile(), Profile())

    with dt[0]:
        im = torch.from_numpy(im).to(model.device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim

    # Inference
    with dt[1]:
        visualize = False #increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=False, visualize=visualize)

    # NMS
    with dt[2]:
        pred = non_max_suppression(pred, conf_thres=0.7, iou_thres=0.75, classes=None, agnostic=False, max_det=1000)

    head_count = 0
    hand_count = 0

    # Process predictions
    for i, det in enumerate(pred):  # per image
        p, im0 = path[i], im0s[i].copy()
        s += f'{i}: '
        annotator = Annotator(im0, line_width=3, example=str(names))
        s = 'PartyAI sees '
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()
            count = {"heads": 0, "hands": 0}
            # Print results
            for c in det[:, 5].unique():
                n = (det[:, 5] == c).sum()  # detections per class
                s += f"{n} {names[int(c)]}, "  # add to string
                count[names[int(c)]] = int(n)
            head_count = count['heads']
            hand_count = count['hands']
            s = s[:-2]
            skipped = add_new_values(head=head_count, hand=hand_count)
            if skipped: 
                lbl.config(text = "Song Skipped!")
            elif count['heads'] > prev_headcount:
                lbl.config(text = "")

            # Write results
            for *xyxy, conf, cls in reversed(det):
                if True:  # Add bbox to image
                    c = int(cls)  # integer class
                    label = None if False else (names[c] if False else f'{names[c]} {conf:.2f}')
                    annotator.box_label(xyxy, label, color=colors(c+8, True))

        # Stream results
        im0 = annotator.result()
        print("FPS: ", round(1/(datetime.datetime.now() - start_time).total_seconds()))

        img = Image.fromarray(im0).resize((640, 640))
        imgtk = ImageTk.PhotoImage(image = img)
        label_widget.imgtk = imgtk
        label_widget.configure(image=imgtk, height = 640, width=640)
        label_widget.after(10, inference, head_count, hand_count)
        tracking_lbl.config(text = s)

# # Create a button to open the camera in GUI app
button1 = Button(app, text="Start Live Stream", command=inference, width=15)
button1.pack()

def addSong2Queue():
    inp = inputtxt.get(1.0, "end-1c")
    add_to_queue(inp)
    lbl.config(text = "Song Queued: "+inp)

# # Create a button to open the camera in GUI app
button2 = Button(app, text="Add Song to Queue", command=addSong2Queue, width=15)
button2.pack()

# TextBox Creation
inputtxt = Text(app,
                   height = 2,
                   width = 15)
  
inputtxt.pack()

# Label Creation
lbl = Label(app, text = "")
lbl.pack()

# Create an infinite loop for displaying app on screen
app.mainloop()

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
1/1: 0...  Success (inf frames 1280x720 at 30.00 FPS)

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/t4/5jwm4m814g5g3vhdxm2lslch0000gn/T/ipykernel_45766/1680756056.py", line 57, in inference
    start_time = datetime.datetime.now()
NameError: name 'datetime' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/t4/5jwm4m814g5g3vhdxm2lslch0000gn/T/ipykernel_45766/1680756056.py", line 57, in inference
    start_time = datetime.datetime.now()
NameError: name 'datetime' is not defined
